In [2]:
from datasets import load_dataset

dataset = load_dataset("ccdv/arxiv-summarization")
dataset

c:\Anaconda\envs\sf2023\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No config specified, defaulting to: arxiv-summarization/section
Found cached dataset arxiv-summarization (C:/Users/Justin Du/.cache/huggingface/datasets/ccdv___arxiv-summarization/section/1.0.0/fa2c9abf4312afb8660ef8e041d576b8e3943ea96ae771bd3cd091b5798e7cc3)
100%|██████████| 3/3 [00:00<00:00,  6.62it/s]


DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 203037
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6436
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6440
    })
})

In [3]:
data_train = dataset['train']
data_val = dataset['validation']
data_test = dataset['test']

In [4]:
from transformers import AutoTokenizer

checkpoint = 'facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


In [5]:
def tokenize_function(data):
    return tokenizer(data['abstract'], data['article'], truncation=True, )

In [6]:
tok = data_test.map(tokenize_function, batched=True)

 86%|████████▌ | 6/7 [00:25<00:04,  4.22s/ba]


In [7]:
tok_val = data_val.map(tokenize_function, batched=True)

Loading cached processed dataset at C:/Users/Justin Du/.cache/huggingface/datasets/ccdv___arxiv-summarization/section/1.0.0/fa2c9abf4312afb8660ef8e041d576b8e3943ea96ae771bd3cd091b5798e7cc3\cache-980c652780991f10.arrow


In [8]:
tok

Dataset({
    features: ['article', 'abstract', 'input_ids', 'attention_mask'],
    num_rows: 6440
})

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [20]:
samples = tok[:2000]
samples = {k: v for k, v in samples.items() if k not in ['abstract', 'article']}
samples_val = tok_val [:2000]
samples = {k: v for k, v in samples_val.items() if k not in ['abstract', 'article']}
batch = data_collator(samples)

In [11]:
print(batch.items())

dict_items([('input_ids', <tf.Tensor: shape=(2000, 1024), dtype=int32, numpy=
array([[    0,   627,   765, ...,   306,    68,     2],
       [    0,  1694,   892, ..., 32027, 24303,     2],
       [    0, 34845,    31, ..., 40051,  1092,     2],
       ...,
       [    0,  1694,  2268, ...,  9285,  2156,     2],
       [    0, 34728,   196, ...,     7,  2268,     2],
       [    0, 10669,   818, ...,   909,  9773,     2]])>), ('attention_mask', <tf.Tensor: shape=(2000, 1024), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])>)])


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [13]:
from transformers import AutoModel
model = AutoModel.from_pretrained(checkpoint)

In [22]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tok,
    eval_dataset=tok_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [23]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartModel.forward` and have been ignored: article, abstract. If article, abstract are not expected by `BartModel.forward`,  you can safely ignore this message.
c:\Anaconda\envs\sf2023\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6440
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2415



AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'new_zeros'

In [ ]:
# chunk = 500
# tok_abs = []

# for i in (data_test[pos:pos + chunk] for pos in range(0, len(data_test), chunk)):
#     tok_abs.append(tokenizer(i['abstract'], truncation=True, padding=True, return_tensors='tf'))


In [ ]:
# tok_art = []
# for i in (data_test[pos:pos + chunk] for pos in range(0, len(data_test), chunk)):
#     tok_art.append(tokenizer(i['article'], truncation=True, ))

In [ ]:
# import torch
# tok = torch.cat(tok_abs, dim=1)


In [ ]:
# test_tok_abs = tokenizer(data_train['article'], truncation=True, padding='max_length')